# Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Victor Harbor,-35.5667,138.6167,64.72,55,100,7.85,AU,overcast clouds,2021-11-07 01:05:26
1,1,Hilo,19.7297,-155.0900,85.28,76,3,3.00,US,clear sky,2021-11-07 01:02:44
2,2,Vaini,-21.2000,-175.2000,82.56,89,75,10.36,TO,broken clouds,2021-11-07 01:05:27
3,3,Leningradskiy,69.3833,178.4167,-5.94,89,3,5.39,RU,clear sky,2021-11-07 01:05:22
4,4,Kapaa,22.0752,-159.3190,84.18,75,40,3.00,US,light rain,2021-11-07 01:05:27


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
1,1,Hilo,19.7297,-155.0900,85.28,76,3,3.00,US,clear sky,2021-11-07 01:02:44
2,2,Vaini,-21.2000,-175.2000,82.56,89,75,10.36,TO,broken clouds,2021-11-07 01:05:27
4,4,Kapaa,22.0752,-159.3190,84.18,75,40,3.00,US,light rain,2021-11-07 01:05:27
8,8,Victoria,22.2855,114.1577,82.45,82,1,1.01,HK,clear sky,2021-11-07 01:03:15
21,21,Bonthe,7.5264,-12.5050,79.74,82,100,3.62,SL,overcast clouds,2021-11-07 01:05:32
22,22,Manado,1.4870,124.8455,81.37,89,40,3.44,ID,scattered clouds,2021-11-07 01:05:33
24,24,Bathsheba,13.2167,-59.5167,80.62,79,46,15.70,BB,scattered clouds,2021-11-07 01:01:07
33,33,Dingle,10.9995,122.6711,83.62,78,92,11.25,PH,overcast clouds,2021-11-07 01:05:36
35,35,Yaan,7.3833,8.5667,75.99,88,100,8.46,NG,light rain,2021-11-07 01:05:36
42,42,Rikitea,-23.1203,-134.9692,75.72,81,96,15.17,PF,overcast clouds,2021-11-07 01:02:07


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                177
City                   177
Lat                    177
Lng                    177
Max Temp               177
Humidity               177
Cloudiness             177
Wind Speed             177
Country                177
Current Description    177
Date                   177
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df=preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                177
City                   177
Lat                    177
Lng                    177
Max Temp               177
Humidity               177
Cloudiness             177
Wind Speed             177
Country                177
Current Description    177
Date                   177
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,85.28,clear sky,19.7297,-155.0900,
2,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
4,Kapaa,US,84.18,light rain,22.0752,-159.3190,
8,Victoria,HK,82.45,clear sky,22.2855,114.1577,
21,Bonthe,SL,79.74,overcast clouds,7.5264,-12.5050,
22,Manado,ID,81.37,scattered clouds,1.4870,124.8455,
24,Bathsheba,BB,80.62,scattered clouds,13.2167,-59.5167,
33,Dingle,PH,83.62,overcast clouds,10.9995,122.6711,
35,Yaan,NG,75.99,light rain,7.3833,8.5667,
42,Rikitea,PF,75.72,overcast clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City                   177
Country                177
Max Temp               177
Current Description    177
Lat                    177
Lng                    177
Hotel Name             177
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))